# Processing

## Setup

In [ ]:
# Python packages
import scanpy as sc
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib

# Settings
sc.settings.verbosity = 0
sc.settings.set_figure_params(dpi=60, facecolor="#FFFFFF", frameon=False, transparent=False)
seed = 10
np.random.seed(seed)

import warnings
warnings.filterwarnings('ignore')

# Custom plotting function
# adapted from https://github.com/scverse/scanpy/issues/2333#issuecomment-1563790561
def split_dim_plot(adata, split_by, embedding="umap", color=None, ncol=5, nrow=None, **kwargs):
    size = 120000 / len(adata.obs_names) * len(adata.obs[split_by].unique()) / 1.2  # Keep point size constant
    categories = adata.obs[split_by].cat.categories
    if nrow is None:
        nrow = int(np.ceil(len(categories) / ncol))
    
    # Update global font sizes
    plt.rcParams.update({
        "font.size": 14,  # General font size
        "axes.titlesize": 18,  # Subplot title size
        "legend.fontsize": 14,  # Legend font size
        "xtick.labelsize": 14,  # X-axis labels
        "ytick.labelsize": 14   # Y-axis labels
    })
    
    fig, axs = plt.subplots(nrow, ncol, figsize=(5*ncol, 5*nrow))
    axs = axs.flatten()

    # Plot individual subplots
    for i, cat in enumerate(categories):
        ax = axs[i]
        
        # Plot background (all cells in light gray)
        sc.pl.embedding(adata, basis=embedding, color=None, size=size, show=False, ax=ax, na_color="#EEEEEE")
        
        # Plot current category with color
        sc.pl.embedding(
            adata[adata.obs[split_by] == cat], 
            basis=embedding, 
            color=color, 
            ax=ax, 
            show=False, 
            title=cat, 
            size=size, 
            legend_loc=None,  # Remove individual legends
            **kwargs
        )
    
        ax.set_title(cat, fontsize=18)  # Set larger font for subplot titles

    # Hide unused subplots if any
    for j in range(i + 1, len(axs)):
        axs[j].axis("off")

    # Create common legend
    if color and color in adata.obs:
        unique_categories = adata.obs[color].cat.categories
        colors = adata.uns.get(color + "_colors", plt.cm.tab10(range(len(unique_categories))))  # Get colors
        
        legend_patches = [plt.Line2D([0], [0], marker="o", color="w", markerfacecolor=c, markersize=12, label=cat)
                          for cat, c in zip(unique_categories, colors)]

        fig.legend(handles=legend_patches, loc="lower center", ncol=min(len(unique_categories), 5), fontsize=16)  # Bigger legend
        unique_categories = adata.obs[color].cat.categories
        colors = adata.uns.get(color + "_colors", plt.cm.tab10(range(len(unique_categories))))  # Get colors
        
        legend_patches = [plt.Line2D([0], [0], marker="o", color="w", markerfacecolor=c, markersize=12, label=cat)
                          for cat, c in zip(unique_categories, colors)]

        fig.legend(handles=legend_patches, loc="lower center", ncol=min(len(unique_categories), 5), fontsize=16)  # Bigger legend

    # Adjust layout to make space for legend
    plt.tight_layout(rect=[0, 0.05, 1, 1])

    # plt.show()

In [ ]:
input_file = "/data/cephfs-1/work/groups/cubi/users/cemo10_c/scRNA/scRNA_preprocessing_pipeline/results/subset/adata_soupX_counts_theislab_tutorial.h5ad"
count_layer = "soupX_counts"
normalization = "log1p_norm"
scale_data_before_pca = False
genes_for_pca = "highly_variable_per_sample" # "all" or "globally_highly_variable" or "highly_variable_per_sample"
cc_method = "None" # None or "regress_out" or "cc_genes_out" or "cc_difference_regressed_out" or "cPCA"
pca_n_components = 5
umap_n_neighbors = 5
output_file = "None"
tsv_file = "/data/cephfs-1/home/users/cemo10_c/work/scRNA/scRNA_preprocessing_pipeline/results/analysis/umap_pca_distances.tsv"
figures_folder = "/data/cephfs-1/home/users/cemo10_c/work/scRNA/scRNA_preprocessing_pipeline/misc/figures_test/"
qc_method = "theislab_tutorial"
save_adata = False

In [ ]:
cc_method = str(cc_method)

# Create folder if it does not exist
import os
if not os.path.exists(figures_folder):
    os.makedirs(figures_folder)

# Path including figure name suffixes
figures_path = str(figures_folder) + str(count_layer) + "-" + str(normalization) + "-" + str(scale_data_before_pca) + "-" + str(genes_for_pca) + "-" + str(cc_method) + "-" + str(pca_n_components) + "-" + str(umap_n_neighbors) + "-" + str(qc_method)

In [ ]:
adata = sc.read_h5ad(input_file, backed = False)
adata.X = adata.layers[normalization + "_of_" + count_layer].copy()
adata

### Analyse cell cycle phases

In [ ]:
cell_cycle_genes = [x.strip() for x in open('/data/cephfs-1/home/users/cemo10_c/work/scRNA/scRNA_preprocessing_pipeline/resources/regev_lab_cell_cycle_genes.txt')]
s_genes = cell_cycle_genes[:43]
g2m_genes = cell_cycle_genes[43:]

cell_cycle_genes = [x for x in cell_cycle_genes if x in adata.var_names]
s_genes = [x for x in s_genes if x in adata.var_names]
g2m_genes = [x for x in g2m_genes if x in adata.var_names]

sc.pp.scale(adata)
sc.tl.score_genes_cell_cycle(adata, s_genes=s_genes, g2m_genes=g2m_genes)
adata.obs["cell_cycle_diff"] = adata.obs["S_score"] - adata.obs["G2M_score"]

if not scale_data_before_pca:
    adata.X = adata.layers[normalization + "_of_" + count_layer].copy()
    print("Not scaling data before PCA")
else:
    print("Scaling data before PCA")

In [ ]:
plt.rcParams['figure.figsize']=(10, 7) # rescale figures
sc.pl.violin(adata, 'total_counts', groupby='sample', size=0, log=True, cut=0, rotation=90)
sc.pl.violin(adata, 'pct_counts_mt', groupby='sample', size=0, rotation=90)

# stacked bar plot of cell cycle phase per sample
cell_cycle = adata.obs.groupby('sample')['phase'].value_counts(normalize=True).unstack().fillna(0)
cell_cycle.plot(kind='bar', stacked=True, figsize=(12, 5), color=['#f8766d', '#00ba38', '#619cff'])
plt.legend(title='phase')
plt.ylabel('fraction of cells')
plt.title('Cell cycle phase distribution')
plt.grid(False)

In [ ]:
# Do this only if we need it later, takes around 6min
# I verified that this method produces similar results to the equivalent R method described in OSCA. The discarded genes in both methods were >90% identical.

if cc_method == "cc_genes_out":
    import statsmodels.api as sm
    from joblib import Parallel, delayed  # Parallel processing
    from tqdm import tqdm  # Progress bar

    sc.pp.highly_variable_genes(adata) # Get top highly variable genes (HVGs)
    sc.tl.pca(adata, n_comps=50, use_highly_variable=True) # Run PCA on the top HVGs
    adata.obs["phase"] = adata.obs["phase"].astype("category") # Ensure 'phase' is categorical
    X = pd.get_dummies(adata.obs["phase"], drop_first=True) # Convert into one-hot encoded variables
    X = sm.add_constant(X)  # Add intercept term
    X = X.astype(float) # Ensure X is numeric
    highly_variable_genes = adata.var["highly_variable"] # Restrict to highly variable genes
    top_hvgs = adata.var_names[highly_variable_genes]

    # Function to compute variance explained for one gene
    def compute_variance(gene):
        y = adata[:, gene].X
        # Convert sparse to dense if needed
        if not isinstance(y, np.ndarray):
            y = y.toarray().flatten()
        y = y.astype(float) # Ensure y is numeric
        model = sm.OLS(y, X).fit() # Fit linear model
        return gene, model.rsquared * 100  # Convert R² to percentage

    # Run in parallel using all available CPU cores (-1)
    results = Parallel(n_jobs=-1)(
        # delayed(compute_variance)(gene) for gene in tqdm(top_hvgs, desc="Processing Genes") # only highly variable
        delayed(compute_variance)(gene) for gene in tqdm(adata.var_names, desc="Processing Genes") # all genes
    )

    variance_by_phase = pd.Series(dict(results)) # Convert results to Pandas Series
    threshold = 5 # arbitrary threshold in percent
    discard = variance_by_phase > threshold # Filter genes where variance explained < threshold
    print(discard.value_counts())  # Summary of genes kept/removed

    # plot the variance explained
    plt.rcParams['figure.figsize']=(13,7) # rescale figures
    plt.hist(variance_by_phase, bins=50)
    plt.axvline(threshold, color="red", linestyle="--")
    plt.xlabel("Variance Explained (%)")
    plt.ylabel("Number of Genes")
    plt.title("Variance Explained by Phase")
    plt.show()

### Perform different cell cycle methods

In [ ]:
plt.rcParams['figure.figsize']=(8,8) # rescale figures to reduce jupyter notebook size
if genes_for_pca == "globally_highly_variable":
    sc.pp.highly_variable_genes(adata)
    print('Using globally highly variable genes')
    sc.pl.highly_variable_genes(adata)

elif genes_for_pca == "highly_variable_per_sample":
    sc.pp.highly_variable_genes(adata, batch_key = "sample")
    print('Using highly variable genes per sample. This simple process avoids the selection of batch-specific genes and acts as a lightweight batch correction method.')
    sc.pl.highly_variable_genes(adata)


if cc_method == "cc_genes_out":
    # adata.var["highly_variable"] = False
    # adata.var.loc[top_hvgs2.flatten(), "highly_variable"] = True
    # adata.var.loc[diff.flatten() > 5, "highly_variable"] = False
    adata.var.loc[discard.index[discard], "highly_variable"] = False
    print('Removing cell cycle genes from highly variable genes')

elif cc_method == "regress_out":
    sc.pp.regress_out(adata, ['S_score', 'G2M_score'])
    print('Regressing out cell cycle scores')

elif cc_method == "cc_difference_regressed_out":
    sc.pp.regress_out(adata, ['cell_cycle_diff'])
    print('Regressing out cell cycle difference score')

elif cc_method == "None":
    print('Not removing cell cycle effects')

# Dimensionality reduction

In [ ]:
# Plotting colors per category
if len(adata.obs['sample'].cat.categories) == 15:
    adata.uns["sample_colors"] = ['#1f77b495', 
                                  '#ff7f0e95', 
                                  '#2ca02c95', 
                                  '#99ff6695', 
                                  '#00ff0095', 
                                  '#8c564b95', 
                                  '#e377c295', 
                                  '#ffff6695', 
                                  '#bcbd2295', 
                                  '#ff000095', 
                                  '#1f77b495',
                                  '#ff7f0e95',
                                  '#2ca02c95',
                                  '#d6272895',
                                  '#00ff9995']

adata.uns["phase_colors"] = ['#ff333370', '#00ff0070', '#0066ff70']
adata.uns["week_colors"] = ['#0066ff70', '#cc00ff70', '#ff993370', '#ff000070', '#00cc0070']
adata.uns['treatment_colors'] = ['#3333cc70', '#00ff0070', '#ff000070']

In [ ]:
if cc_method == "cPCA":
    if genes_for_pca == "all":
        data = adata.X
        bg = adata.X
    else:
        data = adata[:, adata.var['highly_variable']].X
        bg = adata[adata.obs["week"] == "1", adata.var['highly_variable']].X
    
    if not scale_data_before_pca: # convert to dense if needed
        data = data.todense()
        bg = bg.todense()
else:
    # dummy variables for R conversion
    data = ""
    bg = ""

In [ ]:
%load_ext rpy2.ipython

In [ ]:
%%R -i data -i bg -i cc_method -i pca_n_components -o cpca
if(cc_method == "cPCA") {
    library(scPCA)
    cpca = scPCA(
                target=data,
                background=bg,
                penalties=0, 
                n_eigen=pca_n_components, 
                contrasts=200,
                center = FALSE,
                scale = FALSE
                )$x
} else {
    cpca = NULL
}

In [ ]:
# are the counts still integers? (They should be)
print(adata.X[0:5, 0:5])
print(adata.layers['counts'][0:5, 0:5].todense())
adata.raw[0:5, 0:5]

## PCA plots

In [ ]:
if cc_method == "cPCA":
    adata.obsm["X_pca"] = cpca
else:
    sc.tl.pca(adata, n_comps = pca_n_components, use_highly_variable = False if genes_for_pca == "all" else True)
    sc.pl.pca_variance_ratio(adata, log=True, n_pcs = 100)
    sc.pl.pca_loadings(adata)

In [ ]:
plt.rcParams['figure.figsize']=(4,4) # rescale figures to reduce jupyter notebook size
sc.pl.pca(adata, projection = "2d", 
          color = ['sample', 'log1p_total_counts', 'log1p_n_genes_by_counts', 'week', 'treatment', 'phase'],
          wspace = 0.4)

## UMAP plots

In [ ]:
try:
    sc.pp.neighbors(adata, n_pcs=umap_n_neighbors)
except Exception as e:
    print(f"Failed with n_pcs argument: {e}")
    sc.pp.neighbors(adata)
sc.tl.umap(adata, n_components=2, min_dist=0.1, spread=5)

### All samples

In [ ]:
sc.pl.umap(adata,
          color = ['sample', 'log1p_total_counts', 'log1p_n_genes_by_counts', 'week', 'treatment', 'phase'],
          wspace = 0.4)

### Only week 1 of every treatment

In [ ]:
sc.pl.umap(adata[adata.obs["week"] == "1"], color=["treatment"])

Quantify differences between samples and save to a file to compare later.

In [ ]:
from scipy.spatial.distance import pdist, squareform

# Extract PCA coordinates
pca_coords = adata.obsm["X_pca"]

# Compute centroids for each sample
sample_centroids = np.array([
    pca_coords[adata.obs["sample"] == sample].mean(axis=0)
    for sample in ['C_1', 'Conti_1', 'OnOff_1']
])

# Calculate pairwise distances (Euclidean)
pca_distances = squareform(pdist(sample_centroids, metric='euclidean'))
print("Pairwise Euclidean distances between samples:")
print(pca_distances)

In [ ]:
from scipy.spatial.distance import pdist, squareform
import numpy as np

# Extract UMAP coordinates from the AnnData object
umap_coords = adata.obsm["X_umap"]

# Compute centroids for each sample based on UMAP coordinates
# samples = adata.obs["sample"].unique()
sample_centroids = np.array([
    umap_coords[adata.obs["sample"] == sample].mean(axis=0)
    for sample in ['C_1', 'Conti_1', 'OnOff_1']
])

# Calculate pairwise distances using Euclidean metric
umap_distances = squareform(pdist(sample_centroids, metric='euclidean'))
print("Pairwise Euclidean distances between samples (UMAP centroids):")
print(umap_distances)

In [ ]:
import os
from datetime import datetime

dd = umap_distances.flatten()
# save as dataframe and add column names
df = pd.DataFrame([[dd[1], dd[2], dd[5]]], columns=["Conti_1 vs C_1", "OnOff_1 vs C_1", "OnOff_1 vs Conti_1"])
# add column with umap
df["dim_reduc_method"] = ["UMAP"]

# same with pca
dd = pca_distances.flatten()
# save as dataframe and add column names
df_pca = pd.DataFrame([[dd[1], dd[2], dd[5]]], columns=["Conti_1 vs C_1", "OnOff_1 vs C_1", "OnOff_1 vs Conti_1"])
# add column with pca
df_pca["dim_reduc_method"] = ["PCA"]

# combine both dataframes
df = pd.concat([df, df_pca], axis=0)

# add a column with current date and time
df["date"] = datetime.now()

# add columns with parameters used in this notebook
df["input_file"] = input_file
df["normalization"] = normalization
df["scale_data_before_pca"] = scale_data_before_pca
df["genes_for_pca"] = genes_for_pca
df["cc_method"] = cc_method
df["pca_n_components"] = pca_n_components
df["umap_n_neighbors"] = umap_n_neighbors

# save to tsv
# if file does not exist yet, create it and add header
# if the file already exists, append to it
# replace the file if the date if different by more than 1 day

try:
   tsv_file
except NameError:
    tsv_file = "/data/cephfs-1/home/users/cemo10_c/work/scRNA/scRNA_preprocessing_pipeline/results/analysis/umap_pca_distances.tsv"

if not os.path.exists(tsv_file):
    df.to_csv(tsv_file, sep = "\t", index = False)
else:
    df_old = pd.read_csv(tsv_file, sep = "\t")
    print(df_old["date"].max())
    print(datetime.strptime(df_old["date"].max(), '%Y-%m-%d %H:%M:%S.%f'))
    print((pd.Timestamp.now() - datetime.strptime(df_old["date"].min(), '%Y-%m-%d %H:%M:%S.%f')).days)
    if (pd.Timestamp.now() - datetime.strptime(df_old["date"].min(), '%Y-%m-%d %H:%M:%S.%f')).days >= 1:
        df.to_csv(tsv_file, sep = "\t", index = False)
    else:
        df.to_csv(tsv_file, sep = "\t", index = False, mode = "a", header = False)

### All treatments separately

In [ ]:
split_dim_plot(adata, 'treatment', "umap", color='week', ncol=5)

### Only week 1 and 5 of the OnOff treatment

In [ ]:
# subset only treatment OnOff weeks 1 and 5
adata_sub = adata[adata.obs["treatment"] == "OnOff"]
adata_sub = adata_sub[adata_sub.obs["week"].isin(["1", "5"])]
sc.pl.umap(adata_sub, color=["week"], palette = ["#1f77b4", "#ff7f0e"])
del adata_sub

### All samples separately

In [ ]:
split_dim_plot(adata, 'sample', "umap", color = 'phase', ncol = 5)

### Only week 5 of control and OnOff treatment

In [ ]:
adata_sub2 = adata[adata.obs["week"] == "5"]
adata_sub2 = adata_sub2[adata_sub2.obs["treatment"].isin(["Control", "OnOff"])]
sc.pl.umap(adata_sub2, color = ['treatment'])
del adata_sub2

Quantify differences between samples and save to a file to compare later.

In [ ]:
from scipy.spatial.distance import pdist, squareform

# Extract PCA coordinates
pca_coords = adata.obsm["X_pca"]

# Compute centroids for each sample
sample_centroids = np.array([
    pca_coords[adata.obs["sample"] == sample].mean(axis=0)
    for sample in ['OnOff_1', 'OnOff_5']
])

# Calculate pairwise distances (Euclidean)
pca_distances = pdist(sample_centroids, metric='euclidean')
print("Pairwise Euclidean distances between samples:")
print(pca_distances)

from scipy.spatial.distance import pdist, squareform
import numpy as np

# Extract UMAP coordinates from the AnnData object
umap_coords = adata.obsm["X_umap"]

# Compute centroids for each sample based on UMAP coordinates
# samples = adata.obs["sample"].unique()
sample_centroids = np.array([
    umap_coords[adata.obs["sample"] == sample].mean(axis=0)
    for sample in ['OnOff_1', 'OnOff_5']
])

# Calculate pairwise distances using Euclidean metric
umap_distances = pdist(sample_centroids, metric='euclidean')
print("Pairwise Euclidean distances between samples (UMAP centroids):")
print(umap_distances)

import os
from datetime import datetime

dd = umap_distances.flatten()
# save as dataframe and add column names
df = pd.DataFrame([dd], columns=["OnOff_1 vs OnOff_5"])
# add column with umap
df["dim_reduc_method"] = ["UMAP"]

# same with pca
dd = pca_distances.flatten()
# save as dataframe and add column names
df_pca = pd.DataFrame([dd], columns=["OnOff_1 vs OnOff_5"])
# add column with pca
df_pca["dim_reduc_method"] = ["PCA"]

# combine both dataframes
df = pd.concat([df, df_pca], axis=0)

# add a column with current date and time
df["date"] = datetime.now()

# add columns with parameters used in this notebook
df["input_file"] = input_file
df["normalization"] = normalization
df["scale_data_before_pca"] = scale_data_before_pca
df["genes_for_pca"] = genes_for_pca
df["cc_method"] = cc_method
df["pca_n_components"] = pca_n_components
df["umap_n_neighbors"] = umap_n_neighbors


# save to tsv
# if file does not exist yet, create it and add header
# if the file already exists, append to it
# replace the file if the date if different by more than 1 day

import re

tsv_file2 = re.sub("week1_samples", "week1_and_5_onoff_samples.tsv", tsv_file)

if not os.path.exists(tsv_file2):
    df.to_csv(tsv_file2, sep = "\t", index = False)
else:
    df_old = pd.read_csv(tsv_file2, sep = "\t")
    print(df_old["date"].max())
    print(datetime.strptime(df_old["date"].max(), '%Y-%m-%d %H:%M:%S.%f'))
    print((pd.Timestamp.now() - datetime.strptime(df_old["date"].min(), '%Y-%m-%d %H:%M:%S.%f')).days)
    if (pd.Timestamp.now() - datetime.strptime(df_old["date"].min(), '%Y-%m-%d %H:%M:%S.%f')).days >= 1:
        df.to_csv(tsv_file2, sep = "\t", index = False)
    else:
        df.to_csv(tsv_file2, sep = "\t", index = False, mode = "a", header = False)

### Test out different UMAP parameters

In [ ]:
from itertools import product

# Copy adata not to modify UMAP in the original adata object
adata_temp = adata.copy()
# Loop through different umap parameters, recomputting and replotting UMAP for each of them
MIN_DISTS = [0.1, 1, 2]
SPREADS = [0.5, 1, 5]
# Create grid of plots, with a little extra room for the legends
fig, axes = plt.subplots(
    len(MIN_DISTS), len(SPREADS), figsize=(len(SPREADS) * 3 + 2, len(MIN_DISTS) * 3)
)

for (i, min_dist), (j, spread) in product(enumerate(MIN_DISTS), enumerate(SPREADS)):
    ax = axes[i][j]
    param_str = " ".join(["min_dist =", str(min_dist), "and spread =", str(spread)])
    # Recompute UMAP with new parameters
    sc.tl.umap(adata_temp, min_dist=min_dist, spread=spread)
    # Create plot, placing it in grid
    sc.pl.umap(
        adata_temp,
        color=["phase"],
        title=param_str,
        # s=40,
        ax=ax,
        show=False,
    )
plt.tight_layout()
plt.show()
plt.close()
del adata_temp

## t-SNE plots

In [ ]:
try:
    sc.tl.tsne(adata, n_pcs = 50)
except Exception as e:
    print(f"Failed with n_pcs argument: {e}")
    sc.tl.tsne(adata)
sc.pl.tsne(adata, color = 'phase')

In [ ]:
split_dim_plot(adata, 'sample', "tsne", color = 'phase', ncol = 5)

## Diffusion maps

In [ ]:
sc.tl.diffmap(adata)
sc.pl.diffmap(adata, color='phase', components=['1,2','1,3'])

# Save results

In [ ]:
if save_adata:
    adata.write(output_file)

In [ ]:
# save key figures for comparison with other methods
# from https://github.com/scverse/scanpy/issues/1508#issuecomment-734657400

with plt.rc_context():  # Use this to set figure params like size and dpi
    for embedding in ["pca", "umap", "tsne"]:
        for color_by in ['sample', 'week', 'treatment', 'phase']:
            sc.pl.embedding(adata, basis=embedding, color=color_by, wspace=0.4, show=False)
            plt.savefig(figures_path + "-" + embedding + "-combined-" + color_by + ".jpg")
            plt.close()

        split_dim_plot(adata, 'sample', embedding, color='phase', ncol=5)
        plt.savefig(figures_path + "-" + embedding + "-per_sample-" + color_by + ".jpg")
        plt.close()

In [ ]:
# are the counts still integers? (They should be)
print(adata.X[0:5, 0:5])
print(adata.layers['counts'][0:5, 0:5].todense())
adata.raw[0:5, 0:5]